# Compute LD and PCA via PLINK

In this notebook, we compute Linkage Disequilibrium and Principal Components Analysis using [PLINK2](https://www.cog-genomics.org/plink/2.0/).

# Setup 

<div class="alert alert-block alert-warning">
    <b>Cloud Environment</b>: This notebook was written for use on the UK Biobank Research Analysis Platform..
    <ul>
        <li>Use compute type 'Standard VM' with sufficient CPU and RAM (e.g. start with 8 CPUs and 30 GB RAM, increase if needed).</li>
        <li>This notebook can take a while to run. Recommend that it is run in the background via <kbd>dx run</kbd> which will also capture provenance.</li>
    </ul>
</div>

```
dx run dxjupyterlab \
    --instance-type=mem2_ssd1_v2_x8 \
    -icmd="papermill 3_ukb_plink_ld_and_pca.ipynb 3_ukb_plink_ld_and_pca_$(date +%Y%m%d).ipynb" \
    -iin=3_ukb_plink_ld_and_pca.ipynb \
    --folder=outputs/plink-ld-pca/$(date +%Y%m%d)/
```

In [ ]:
import os
import pandas as pd

## Setup plink2

https://www.cog-genomics.org/plink/2.0/

In [ ]:
%%bash

##### plink 2 install
PLINK_VERSION=2.3.Alpha
PLINK_ZIP_PATH=/tmp/plink-$PLINK_VERSION.zip
curl -L -o $PLINK_ZIP_PATH https://s3.amazonaws.com/plink2-assets/alpha2/plink2_linux_x86_64.zip
mkdir -p /tmp/plink2/
unzip -o $PLINK_ZIP_PATH -d /tmp/plink2/

In [ ]:
!/tmp/plink2/plink2 --version # --help

## Define constants

This takes as input the WES data from UK Biobank further filtered by notebook `1_ukb_plink_bed_filter.ipynb` and then merged by notebook `2_ukb_plink_merge_bed_files.ipynb`.

In [ ]:
BED_FILE = '/mnt/project/outputs/plink-merge-bed/20211028/ukb_200kwes_filtered_plink_mergebed'

# Outputs
PLINK_OUTPUT_FILENAME_PREFIX = 'ukb_200kwes_lipids'

# Compute Linkage Disequilibrium via PLINK

In [ ]:
!/tmp/plink2/plink2 \
  --bfile {BED_FILE} \
  --chr 1-22 \
  --indep-pairwise 200 50 0.25 \
  --out {PLINK_OUTPUT_FILENAME_PREFIX}_plink_ld

In [ ]:
%%bash

ls -lat | head

In [ ]:
%%bash

wc -l *prune*

# Compute Principal Components Analysis via PLINK

<div class="alert alert-block alert-warning">
    <b>Note</b>: the <kbd>--memory</kbd> parameter below assumes the machine has 30 GB of RAM. Adjust this value if the machine has more or less than 30 GB of RAM.
</div>

In [ ]:
!/tmp/plink2/plink2 \
  --bfile {BED_FILE} \
  --chr 1-22 \
  --pca 15 approx \
  --memory 27500 \
  --out {PLINK_OUTPUT_FILENAME_PREFIX}_plink_pca

In [ ]:
%%bash

ls -lat | head

# Provenance 

In [ ]:
%%bash

date

In [ ]:
%%bash

pip3 freeze